# TP 2 : Modélisation de la sécheresse agricole en Côte d'Ivoire

## Cas de l'événement 2016 (Nord ivoirien) — Formation CLIMADA DGE Côte d'Ivoire

**Objectifs du TP :**
- Créer un hazard sécheresse à partir de données météo mensuelles de 2016
- Modéliser l'exposition agricole régionale (coton, riz, maïs)
- Calculer l'impact économique de la sécheresse par culture
- Comparer les résultats à la réalité rapportée
- Analyse de scénarios d'adaptation (variétés tolérantes, micro-irrigation, assurance)

**Durée estimée :** 3h

**Données utilisées :**
- Précipitations, SPI et anomalies température 2016 (KORHOGO, BOUAKE, BONDOUKOU, FERKESSEDOUGOU)
- Valeur agricole par culture/zone
- Exposition: [exposition_agricole_nord_ci.csv]
- Aléa: [secheresse_agriculture_2016.csv]

---

## 1. Configuration de l'environnement et import des bibliothèques


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
from climada.hazard import Hazard
from climada.entity import Exposures, ImpactFunc, ImpactFuncSet
from climada.engine import Impact
from climada.util.coordinates import get_grid_points


## 2. Importation et exploration des données agro-météorologiques 2016


In [ ]:
# Importation des données
drought_df = pd.read_csv('secheresse_agriculture_2016.csv', parse_dates=['date'])
print('Stations disponibles:', drought_df['station'].unique())
drought_df.head()

### Analyse SPI mensuel
Affichons l'évolution du SPI sur 4 stations principales et identifions la période la plus sèche.


In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
for station in drought_df['station'].unique():
    data = drought_df[drought_df['station']==station]
    ax.plot(data['date'], data['spi_3m'], marker='o', label=station)
ax.axhline(y=-1, color='red', linestyle='--', label='Seuil sécheresse modérée')
ax.axhline(y=-1.5, color='orange', linestyle='--', label='Sécheresse sévère')
ax.axhline(y=-2, color='purple', linestyle='-.', label='Sécheresse extrême')
plt.title('Évolution SPI 3 mois en 2016 (Nord CI)')
plt.ylabel('SPI')
plt.legend()
plt.grid(alpha=0.3)
plt.show()


## 3. Création de l'aléa sécheresse avec CLIMADA


In [ ]:
# Centroids région Nord, grille 10 km
north_bounds = {'min_lat': 7.5, 'max_lat': 10.5, 'min_lon': -7.5, 'max_lon': -2.5}
centroids_nord = get_grid_points(north_bounds['min_lat'], north_bounds['min_lon'],
    north_bounds['max_lat'], north_bounds['max_lon'], res_km=10)
n_centroids = len(centroids_nord)

# Sélection des sécheresses modérées à extrêmes
drought_events = drought_df[drought_df['spi_3m']<=-1]
n_events = len(drought_events)

from scipy import sparse
# Création matrice intensité
intensity_matrix = np.zeros((n_events, n_centroids))
# Attribution SPI par station aux centroids géographiquement proches
for i, (_, evt) in enumerate(drought_events.iterrows()):
    dist = [np.sqrt((evt['latitude']-lat)**2+(evt['longitude']-lon)**2) for lat,lon in centroids_nord]
    weights = np.exp(-np.array(dist)/1.0)
    intensity_matrix[i] = evt['spi_3m'] * weights / weights.sum()

hazard_drought = Hazard()
hazard_drought.tag = {'haz_type': 'DR', 'description': 'Sécheresse Nord 2016'}
hazard_drought.units = 'SPI'
hazard_drought.centroids.set_lat_lon(centroids_nord[:,1], centroids_nord[:,0])
hazard_drought.event_id = np.arange(1, n_events+1)
hazard_drought.date = drought_events['date'].values
hazard_drought.frequency = [1] * n_events
hazard_drought.intensity = sparse.csr_matrix(intensity_matrix)
hazard_drought.check()
print('✅ Hazard sécheresse créé')

## 4. Modélisation de l'exposition agricole


In [ ]:
expo_df = pd.read_csv('exposition_agricole_nord_ci.csv')
expo_df


Création GeoDataFrame Exposures pour CLIMADA


In [ ]:
geometry = [Point(np.random.uniform(north_bounds['min_lon'], north_bounds['max_lon']),
    np.random.uniform(north_bounds['min_lat'], north_bounds['max_lat'])) for _ in range(len(expo_df))]
expo_gdf = gpd.GeoDataFrame(expo_df, geometry=geometry)
exposures_agri = Exposures(expo_gdf)
exposures_agri.gdf['category_id'] = 2
exposures_agri.gdf['region_id'] = np.arange(1,len(expo_df)+1)
exposures_agri.gdf['impf_DR'] = 1
print('Exposition agricole CLIMADA créée')

## 5. Définition de fonctions de dommage agricoles (exemple coton)


In [ ]:
impact_func_coton = ImpactFunc()
impact_func_coton.id = 1
impact_func_coton.name = 'Coton_sécheresse_CI'
impact_func_coton.intensity_unit = 'SPI'
impact_func_coton.haz_type = 'DR'
intensities = np.array([0, 1, 1.5, 2, 2.5, 3, 4])
losses = np.array([0, 0.1, 0.25, 0.4, 0.6, 0.75, 0.9])  # 90% perte rendement extrême
impact_func_coton.intensity = intensities
impact_func_coton.mdd = losses 
impact_func_coton.paa = np.ones_like(losses) 
impact_func_set = ImpactFuncSet()
impact_func_set.append(impact_func_coton)
impact_func_set.check()
print('Fonction impact coton OK')

## 6. Calcul de l'impact économique


In [ ]:
impact_coton = Impact()
impact_coton.calc(exposures=exposures_agri, impact_funcs=impact_func_set, hazard=hazard_drought)
print('Perte annuelle moyenne (coton):', int(impact_coton.aai_agg/1e6), 'millions FCFA')
print('Impact max événement:', int(impact_coton.at_event.max()/1e6), 'millions FCFA')

## 7. Analyse et synthèse
Comparez le total des dommages estimés à la valeur du secteur, commentez.

Proposez un scénario d'adaptation et ré-estimez l'impact.
